In [1]:
import tensorflow as tf

In [2]:
mnist = tf.keras.datasets.mnist
(x_train, y_train),(x_test,y_test) = mnist.load_data()

x_train , x_test = x_train/255.0, x_test/255.0

mlp_model = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=(28,28)),
    tf.keras.layers.Dense(128,activation='relu'),
    tf.keras.layers.Dense(10,activation='softmax')
])

mlp_model.compile(optimizer='adam', loss = 'sparse_categorical_crossentropy', metrics=['accuracy'])
mlp_model.fit(x_train,y_train,epochs=5)

Epoch 1/5
1875/1875 [==============================] - 3s 1ms/step - loss: 0.2568 - accuracy: 0.9271
Epoch 2/5
1875/1875 [==============================] - 3s 1ms/step - loss: 0.1126 - accuracy: 0.9668
Epoch 3/5
1875/1875 [==============================] - 3s 1ms/step - loss: 0.0771 - accuracy: 0.9772
Epoch 4/5
1875/1875 [==============================] - 3s 1ms/step - loss: 0.0586 - accuracy: 0.9825
Epoch 5/5
1875/1875 [==============================] - 3s 1ms/step - loss: 0.0441 - accuracy: 0.9862


In [3]:
mlp_model.evaluate(x_test,y_test)

313/313 [==============================] - 0s 959us/step - loss: 0.0735 - accuracy: 0.9796


[0.0735325962305069, 0.9796000123023987]

In [4]:
converter = tf.lite.TFLiteConverter.from_keras_model(mlp_model)
tflite_model = converter.convert()

INFO:tensorflow:Assets written to: C:\Users\Public\Documents\ESTsoft\CreatorTemp\tmpmnjjjywj\assets


INFO:tensorflow:Assets written to: C:\Users\Public\Documents\ESTsoft\CreatorTemp\tmpmnjjjywj\assets


In [5]:
with open('./mnist_model.tflite','wb') as f:
    f.write(tflite_model)

In [6]:
# tf.saved_model.save(mlp_model,"./mlp_model/")

In [7]:
mlp_model.save("./mlp_model2/")

INFO:tensorflow:Assets written to: ./mlp_model2/assets


INFO:tensorflow:Assets written to: ./mlp_model2/assets


In [8]:
saved_model = tf.keras.models.load_model("./mlp_model2/")

In [9]:
converter = tf.lite.TFLiteConverter.from_saved_model("./mlp_model2/")
tflite_model = converter.convert()

In [10]:
with open('./saved_model.tflite','wb') as f:
    f.write(tflite_model)

In [11]:
class Inc(tf.keras.layers.Layer):
    def call(self, inputs):
        return inputs + 1
inc = Inc()

In [12]:
class Inc_Graph(tf.keras.layers.Layer):
    @tf.function
    def call(self, inputs):
        return inputs + 1
inc_g = Inc_Graph()

In [13]:
inc_g2 = tf.function(inc)

In [14]:
print(inc_g(tf.constant(3)))
print(inc_g(tf.constant([3,2])))

tf.Tensor(4, shape=(), dtype=int32)
tf.Tensor([4 3], shape=(2,), dtype=int32)


In [15]:
concrete_fun = inc_g.call.get_concrete_function(tf.TensorSpec(shape=(1,3),dtype=tf.float32))

In [16]:
print(concrete_fun(tf.constant([1.0,2.0,3.0])))

tf.Tensor([2. 3. 4.], shape=(3,), dtype=float32)


In [17]:
mlp_model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28,28)),
    tf.keras.layers.Dense(128,activation='relu'),
    tf.keras.layers.Dense(10,activation='softmax')
])

In [18]:
graph_model = tf.function(mlp_model)
concrete_function = tf.function(mlp_model)
concrete_func = graph_model.get_concrete_function(tf.TensorSpec(shape=mlp_model.inputs[0].shape,dtype=mlp_model.input[0].dtype))

In [19]:
class MLP_Model(tf.keras.Model):
    def __init__(self):
        super(MLP_Model, self).__init__()
        self.flatten = tf.keras.layers.Flatten()
        self.dense = tf.keras.layers.Dense(128, activation='relu')
        self.softmax = tf.keras.layers.Dense(10, activation='softmax')
        
    @tf.function
    def call(self, inputs):
        x = self.flatten(inputs)
        x = self.dense(x)
        return self.softmax(x)

# Create an instance of the model
mlp_model = MLP_Model()

# Define input shape and dtype
input_shape = (None, 28, 28)
input_dtype = tf.float32

# Create a concrete function using the model's input signature
concrete_func = mlp_model.call.get_concrete_function(tf.TensorSpec(shape=input_shape, dtype=input_dtype))

# Print the concrete function
print(concrete_func)


ConcreteFunction call(inputs)
  Args:
    inputs: float32 Tensor, shape=(None, 28, 28)
  Returns:
    float32 Tensor, shape=(None, 10)
